In [91]:
import json

import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.feature_extraction import text
from sklearn.metrics import silhouette_score


with open('aneks.json') as file:
    aneks = json.loads(file.read())

full = []
source = []
names_aneks={}







In [92]:
import pymorphy2
from sklearn.feature_extraction import text
morph = pymorphy2.MorphAnalyzer(lang='ru')
import nltk
from nltk.tokenize import TweetTokenizer


def normalize(text_to_n):
    global names_aneks
    text_to_n = text_to_n.replace('анек', 'анекдот').replace('-', ' ')
    tokenizer = TweetTokenizer()
    result = ""
    names = set()
    words = tokenizer.tokenize(text_to_n)
    stopwords = text.ENGLISH_STOP_WORDS.union(nltk.corpus.stopwords.words('russian')+['это', 'тебе', '.', ',', ':', '-', '!', '?', '"', '\'', 'и', 'а'])
    for word in words:
        if word not in stopwords:
            parsed = morph.parse(word)[0]
            if parsed.tag.animacy == "anim":
                names.add(parsed.normal_form)
                result += parsed.normal_form+" "
    for elem in names:
        if elem not in names_aneks:
            names_aneks[elem]=1
        else:
            names_aneks[elem]+=1
    return result

In [93]:

parsed = morph.parse("мама")[0]
print(parsed.tag.animacy)


anim


In [94]:
full = []
full_dubl = []
source = []
source_dubl = []
names_aneks={}
for anek in aneks:
        if ("attachments" not in anek) and anek['text']!='':
            if len(anek['text'])>20:
                full_dubl.append(normalize(anek['text']))
                source_dubl.append(anek['text'])
print(len(full_dubl))
for i in range(len(full_dubl)):
    if full_dubl[i] not in full:
        full.append(full_dubl[i])
        source.append(source_dubl[i])
        
print(len(full))
                


vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(full)



10835
7819


In [95]:
models = {}

In [100]:
len(names_aneks)
sorted_names = sorted(names_aneks.items(), key=lambda x:x[1], reverse=True)
for name in sorted_names[:200]:
    print("{} - {}".format(name[0], name[1]))

мужик - 1301
жена - 613
человек - 580
девушка - 449
друг - 387
мама - 382
женщина - 376
муж - 357
сын - 351
папа - 343
доктор - 336
ребёнок - 332
отец - 289
мужчина - 283
мальчик - 273
парень - 258
врач - 231
член - 226
мать - 222
сынок - 201
девочка - 195
дед - 190
бог - 178
еврей - 178
товарищ - 175
пол - 160
че - 158
бабушка - 154
грузин - 149
баба - 148
бармен - 129
собака - 124
петрович - 122
иванович - 118
женя - 116
сосед - 113
солдат - 111
медведь - 107
студент - 105
сука - 105
негр - 104
лицо - 103
пьяный - 102
вовочка - 100
дочь - 99
брат - 99
бабка - 95
лошадь - 94
поручик - 93
господин - 91
дедушка - 90
хозяин - 89
волк - 89
гость - 89
дочка - 88
водитель - 87
сэр - 85
заяц - 85
немец - 84
господь - 84
американец - 83
француз - 83
учитель - 83
василий - 80
вася - 79
учительница - 79
родитель - 77
мент - 77
иван - 77
ребята - 77
пидор - 76
рыба - 76
дядя - 76
петька - 76
продавец - 75
свинья - 72
старик - 72
проститутка - 71
чукча - 69
путин - 69
пациент - 67
дурак - 67
конь

In [99]:

for true_k in range(20, 240,3):
    models[true_k] = KMeans(n_clusters=true_k, ).fit(X)
    label = models[true_k].labels_
    sil_coeff = silhouette_score(X, label, metric='euclidean')
    print("For n_clusters={}, The Silhouette Coefficient is {}".format(true_k, sil_coeff))

For n_clusters=20, The Silhouette Coefficient is 0.03439729025146093
For n_clusters=23, The Silhouette Coefficient is 0.03822239570866091
For n_clusters=26, The Silhouette Coefficient is 0.0428223510343968
For n_clusters=29, The Silhouette Coefficient is 0.04019216507549579
For n_clusters=32, The Silhouette Coefficient is 0.04708246116215939
For n_clusters=35, The Silhouette Coefficient is 0.048327187285992924
For n_clusters=38, The Silhouette Coefficient is 0.048507397407291424
For n_clusters=41, The Silhouette Coefficient is 0.049064566133030355
For n_clusters=44, The Silhouette Coefficient is 0.05010807868190061
For n_clusters=47, The Silhouette Coefficient is 0.054169833703995286
For n_clusters=50, The Silhouette Coefficient is 0.056569579789820953
For n_clusters=53, The Silhouette Coefficient is 0.060087473076273844
For n_clusters=56, The Silhouette Coefficient is 0.05775835049228479
For n_clusters=59, The Silhouette Coefficient is 0.05830605219464429


KeyboardInterrupt: 

In [75]:

true_k = 300
model = models[true_k]

print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]

terms = vectorizer.get_feature_names()
for i in range(true_k):
    print ("Cluster %d:" % i)
    for ind in order_centroids[i, :15]:
        print(' %s' % terms[ind])

clusters = model.predict(X)

for cluster in range(true_k):
    file = open('{}.txt'.format(cluster), 'w')
    print('****'*10)
    print("Cluster {}".format(cluster))
    s = 0
    for i in range(len(full)):
            
            if clusters[i] == cluster:
                s+=1
                file.write(source[i])
                file.write('\n\n\n\n\n')
            
        
    print(s)
    file.close()
                

Top terms per cluster:
Cluster 0:
 муж
 женщина
 жена
 женя
 мужик
 доктор
 любовник
 ребёнок
 врач
 дочка
 сексолог
 гинеколог
 электрик
 супруг
 психиатр
Cluster 1:
 пися
 член
 мужик
 мама
 сыночек
 мент
 ребёнок
 панымаеш
 кузнецов
 нарушаэма
 орлов
 бабушка
 вова
 дед
 сынок
Cluster 2:
 сынок
 отец
 сын
 папа
 батя
 мамка
 пират
 ишак
 бог
 дебил
 кобыла
 взрослый
 пидарас
 молодец
 дед
Cluster 3:
 батя
 сын
 дочь
 отец
 маманя
 мальчик
 родитель
 медведь
 вовочка
 чеченец
 уазик
 жлоб
 парень
 кароч
 мужик
Cluster 4:
 женщина
 мужик
 мужчина
 грузин
 журналист
 вратарь
 мультименструациалистка
 спелеолог
 рабочий
 будт
 эксгибиционист
 поpог
 фаллоимитатор
 улитка
 баранов
Cluster 5:
 женя
 мужик
 пьяный
 муж
 сократ
 родственник
 нина
 навозник
 врач
 жук
 жена
 мама
 михаил
 механик
 супруга
Cluster 6:
 мама
 ребёнок
 папа
 сын
 мальчик
 дочка
 дочь
 девочка
 женщина
 вовочка
 вова
 маша
 сыночек
 мать
 молодец
Cluster 7:
 доктор
 член
 врач
 уролог
 мужик
 батенька
 теоретик
 

In [30]:
for i in range(3):
    file = open('{}.txt'.format(i), 'w')
    file.write(full[i])
    file.write('\n\n\n\n')
    file.close()

For n_clusters=7, The Silhouette Coefficient is 0.010061492880524483
For n_clusters=8, The Silhouette Coefficient is 0.010920030541936125
For n_clusters=9, The Silhouette Coefficient is 0.011205732753257733
For n_clusters=10, The Silhouette Coefficient is 0.011409887543775784
For n_clusters=11, The Silhouette Coefficient is 0.011877082976872193
For n_clusters=12, The Silhouette Coefficient is 0.012034316272928511
For n_clusters=13, The Silhouette Coefficient is 0.013062573266366295
For n_clusters=14, The Silhouette Coefficient is 0.013088960566806345
For n_clusters=15, The Silhouette Coefficient is 0.013373067097145249
For n_clusters=16, The Silhouette Coefficient is 0.013940559815239683
For n_clusters=17, The Silhouette Coefficient is 0.01480341096653649
For n_clusters=18, The Silhouette Coefficient is 0.01455557370823081
For n_clusters=19, The Silhouette Coefficient is 0.01496585728700298
For n_clusters=20, The Silhouette Coefficient is 0.015373245533247915
For n_clusters=21, The Silhouette Coefficient is 0.015263296743845437
For n_clusters=22, The Silhouette Coefficient is 0.015895298648375328
For n_clusters=23, The Silhouette Coefficient is 0.016112882458433155
For n_clusters=24, The Silhouette Coefficient is 0.015975250361236676
For n_clusters=25, The Silhouette Coefficient is 0.01616270620354599
For n_clusters=26, The Silhouette Coefficient is 0.015605088939544112
For n_clusters=27, The Silhouette Coefficient is 0.01785271519630537
For n_clusters=28, The Silhouette Coefficient is 0.017339685162316525
For n_clusters=29, The Silhouette Coefficient is 0.018319018672883127
For n_clusters=30, The Silhouette Coefficient is 0.018421742848094967
For n_clusters=31, The Silhouette Coefficient is 0.01868218631890221
For n_clusters=32, The Silhouette Coefficient is -0.004095727113227382
For n_clusters=33, The Silhouette Coefficient is 0.01804964558591296
For n_clusters=34, The Silhouette Coefficient is 0.018650022372136314
For n_clusters=35, The Silhouette Coefficient is 0.019156552206185846
For n_clusters=36, The Silhouette Coefficient is 0.01795673281475781
For n_clusters=37, The Silhouette Coefficient is 0.021060246828426433
For n_clusters=38, The Silhouette Coefficient is 0.02026362902456489
For n_clusters=39, The Silhouette Coefficient is 0.02137843170772625
For n_clusters=40, The Silhouette Coefficient is 0.0201536533518736
For n_clusters=41, The Silhouette Coefficient is 0.020351825888301287
For n_clusters=42, The Silhouette Coefficient is 0.02058212943945563
For n_clusters=43, The Silhouette Coefficient is -0.006767131632652018
For n_clusters=44, The Silhouette Coefficient is 0.022420318258019937
For n_clusters=45, The Silhouette Coefficient is 0.021965530746545826
For n_clusters=46, The Silhouette Coefficient is 0.020036476847443703
For n_clusters=47, The Silhouette Coefficient is 0.02055232368525555
For n_clusters=48, The Silhouette Coefficient is 0.02167008063895293
For n_clusters=49, The Silhouette Coefficient is 0.023081063755350014
For n_clusters=50, The Silhouette Coefficient is 0.021635993178514584
For n_clusters=51, The Silhouette Coefficient is -0.004460050418390375
For n_clusters=52, The Silhouette Coefficient is 0.02281362672992794
For n_clusters=53, The Silhouette Coefficient is 0.022263294209234005
For n_clusters=54, The Silhouette Coefficient is 0.021923787639207777
For n_clusters=55, The Silhouette Coefficient is -0.011403821377354113
For n_clusters=56, The Silhouette Coefficient is 0.023337317071652697
For n_clusters=57, The Silhouette Coefficient is 0.022558970368341733
For n_clusters=58, The Silhouette Coefficient is 0.024344885915585954
For n_clusters=59, The Silhouette Coefficient is -0.009895913227486416
For n_clusters=60, The Silhouette Coefficient is 0.022864246275411095